In [ ]:
import csv
from datasets import load_dataset, Dataset, concatenate_datasets, load_from_disk

In [ ]:
with open('../webui/non_corruption.csv') as csvfile:
    reader = csv.DictReader(csvfile, fieldnames=['url', 'title', 'description', 'text', 'date', 'keywords'])
    rows = list(reader)

In [ ]:
non_corruption_dataset = Dataset.from_list(rows)

In [ ]:
non_corruption_dataset = non_corruption_dataset.map(
    lambda row: {
        'label': 0,
        'is_hand_annoted': True,
        'keywords': row['keywords'].split(',')
    }
)

In [ ]:
non_corruption_dataset

In [ ]:
dataset = load_dataset('boapps/kmdb_classification')

In [ ]:
dataset = dataset.remove_columns(['authors', 'kmonitor_description', 'kmonitor_title', 'kmonitor_tags', 'kmonitor_institutes', 'kmonitor_people', 'kmonitor_places', 'kmonitor_collections'])

In [ ]:
dataset

In [ ]:
dataset['train']

In [ ]:
dataset['train'] = concatenate_datasets([dataset['train'], non_corruption_dataset])

In [ ]:
import re

html_pattern = re.compile(r'<.*?>')
date_pattern = re.compile(r'\d{4}\-\d{2}\-\d{2} \d{2}:\d{2}')
slug_pattern = re.compile(r' \[sharedcontent .*\]')

In [ ]:
def clean_article(article):
    article['description'] = html_pattern.sub('', article['description'])
    article['description'] = date_pattern.sub('', article['description'])
    article['description'] = slug_pattern.sub('', article['description'])
    article['description'] = article['description'].replace(r'\xa0', ' ')
    article['description'] = article['description'].replace(r'Ã¶', 'ö')
    article['description'] = article['description'].replace(r'Ã©', 'é')
    article['description'] = article['description'].replace(r'Ã³', 'ó')
    article['description'] = article['description'].replace(r'Ã¡', 'á')
    article['description'] = article['description'].replace(r'Ã\xad', 'í')
    article['description'] = article['description'].replace(r'Ã¼', 'ü')
    article['description'] = article['description'].replace(r'Ã¼', 'ü')
    article['description'] = article['description'].replace(r'+ Videó', '')
    article['description'] = article['description'].replace(r'•', '')
    article['description'] = article['description'].replace(r'MNO - PL', ' ')
    article['description'] = article['description'].replace('\\\\', '')
    article['description'] = article['description'].strip('- ')

    article['title'] = article['title'].replace(r'Belföld :: ', '')
    article['title'] = article['title'].replace(r' (+videó)', '')
    article['title'] = article['title'].replace('\\\\', '')

    return article

In [ ]:
dataset = dataset.map(clean_article)

In [ ]:
dataset = dataset.filter(lambda row: row['description'] != '')

In [ ]:
RE_SUSPICIOUS = re.compile(r'[#<>{}\|\=\*\[\]\\]')
field = 'description'
dataset = dataset.map(lambda row: {'score': len(RE_SUSPICIOUS.findall(row[field]))/len(row[field])})
dataset = dataset.map(lambda row: {'title_score': len(RE_SUSPICIOUS.findall(row['title']))/len(row['title'])})

In [ ]:
dataset.filter(lambda row: row['title_score'] > 0.01)['train']['title']

In [ ]:
dataset = dataset.filter(lambda row: row['score'] < 0.01)
dataset = dataset.filter(lambda row: row['title_score'] < 0.01)

In [ ]:
dataset

In [ ]:
dataset.shuffle(seed=42).save_to_disk('../datasets/base/kmdb_classification_extended')

In [ ]:
classification_dataset = load_from_disk('../datasets/base/kmdb_classification_extended')

In [ ]:
classification_dataset